In [1]:
import sys
import re
import time

import requests
from bs4 import BeautifulSoup

In [13]:
url = 'https://paperswithcode.com/datasets?mod=videos&page=1'
res = requests.get(url)

soup = BeautifulSoup(res.text, 'html.parser')
title = soup.find_all(class_='filter-items')

counter = 0
title_list = []
while True:
    try:
        text = title[1].contents[counter].get_text(strip=True)
        if text != '':
            text = re.sub(r'\d*$', '', text)
            title_list.append(text)
        counter += 1
    except IndexError:
        break

len(title_list)

499

In [15]:
from selenium import webdriver
from selenium.webdriver.common.by import By

browser = webdriver.Chrome()

# ヘッドレスモード
# from selenium.webdriver.chrome.options import Options
# options = Options()
# options.add_argument('--headless')
# browser = webdriver.Chrome(options=options)


browser.set_window_size('1200', '1000')
browser.get('https://paperswithcode.com/')

obj = []
not_page =[]

In [16]:
for i in range(len(title_list)):

    sys.stdout.write(f'\033[2K\033[{i}/{len(title_list)}]')
    sys.stdout.flush()

    task_obj = {}
    task_obj['task'] = title_list[i]

    box = browser.find_element(By.NAME, 'q')
    box.send_keys(task_obj['task'])
    box.submit()

    try:
        description = browser.find_element(By.CLASS_NAME, 'description')
        task_obj['script'] = description.text
        task_obj['url'] = browser.current_url
        obj.append(task_obj)

    except:
        error_id = {'len_id': i, 'task': task_obj['task']}
        not_page.append(error_id)
        box = browser.find_element(By.NAME, 'q')
        box.clear()

    time.sleep(1)

[0/499][1/499][2/499][3/499][4/499][5/499][6/499][7/499][8/499][9/499][10/499][11/499][12/499][13/499][14/499][15/499][16/499][17/499][18/499][19/499][20/499][21/499][22/499][23/499][24/499][25/499][26/499][27/499][28/499][29/499][30/499][31/499][32/499][33/499][34/499][35/499][36/499][37/499][38/499][39/499][40/499][41/499][42/499][43/499][44/499][45/499][46/499][47/499][48/499][49/499][50/499][51/499][52/499][53/499][54/499][55/499][56/499][57/499][58/499][59/499][60/499][61/499][62/499][63/499][64/499][65/499][66/499][67/499][68/499][69/499][70/499][71/499][72/499][73/499][74/499][75/499][76/499][77/499][78/499][79/499][80/499][81/499][82/499][83/499][84/499][85/499][86/499][87/499][88/499][89/499][90/499][91/499][92/499][93/499][94/499][95/499][96/499][97/499][98/499][99/499][100/499][101/499][102/499][103/499][104/499][105/499][106/499][107/499][108/499][109/499][110/499]

In [28]:
import pandas as pd
descriptions = pd.DataFrame(obj)
display(descriptions)
descriptions.to_csv('./description.csv')

,task,script,url
0,Action Recognition,Action Recognition is a computer vision task t...,https://paperswithcode.com/task/action-recogni...
1,Temporal Action Localization,Temporal Action Localization aims to detect ac...,https://paperswithcode.com/task/action-recogni...
2,Object Detection,Object Detection is a computer vision task in ...,https://paperswithcode.com/task/object-detection
3,Object Tracking,Object tracking is the task of taking an initi...,https://paperswithcode.com/task/object-tracking
4,Video Understanding,A crucial task of Video Understanding is to re...,https://paperswithcode.com/task/video-understa...
...,...,...,...
494,Portrait Segmentation,This task has no description! Would you like t...,https://paperswithcode.com/task/portrait-segme...
495,Pose Contrastive Learning,Please enter a description about the method he...,https://paperswithcode.com/method/pose-contras...
496,Privacy Preserving Deep Learning,The goal of privacy-preserving (deep) learning...,https://paperswithcode.com/task/privacy-preser...
497,drone-based object tracking,drone-based object tracking,https://paperswithcode.com/task/drone-based-ob...
